In [56]:
#importing all the libraries 
import numpy as np 
import pandas as pd 
import os 
import joblib as jb
from collections import defaultdict
from itertools import groupby
from math import sqrt, atan2
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from tensorflow.keras.layers import Input, Flatten, Dropout, Dense, Concatenate
from tensorflow.keras.models import Model
from tcn import TCN
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss


In [4]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pratham21082 (pratham_03). Use `wandb login --relogin` to force relogin


True

In [5]:
wandb.init(project = "ISAC_2_TCN",name="Phase1_ph_new (5 epochs)")

In [24]:
path = '/home/vijay/paper_jc/conference/phase1_ph_student_label'
all_files=[]
for subfolder in os.listdir(path):
        subfolder_path = os.path.join(path, subfolder)
        if os.path.isdir(subfolder_path):
            files = os.listdir(subfolder_path)
            files.sort()
            for file in files:
                file_path = os.path.join(subfolder_path, file)
                df = pd.read_csv(file_path, index_col=0)
                all_files.append(df)

    

In [26]:
df = pd.concat(all_files, axis=0 )
df  = df.reset_index(drop=True)
print(df.isnull().values.any())
Y = df['headlabel']
X = df.iloc[0:,0:-1]
selector = SelectKBest(f_classif, k=25)
csi_feature_25 = selector.fit_transform(X, Y)

False


In [27]:
csi_feature_5X5=np.reshape(csi_feature_25,(csi_feature_25.shape[0],5,5))
csi_feature_5X5.shape

(1970832, 5, 5)

In [28]:
Y = pd.DataFrame(Y)
onehot_encoder = OneHotEncoder()
csi_headlabel= onehot_encoder.fit_transform(Y)

In [29]:

X_train, X_ , y_train,y_ = train_test_split(csi_feature_5X5, csi_headlabel, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val=train_test_split(X_, y_, test_size=0.5, random_state=42)

In [44]:
def csi_network_inc_res(input_sh, output_sh):
    # input_shape = (340, 100)
    nb_filters = 64
    x_input = Input(input_sh)
    tcn1 = TCN(
        nb_filters=nb_filters,
        nb_stacks = 1,
        kernel_size=10,
        dilations=(1, 2, 4, 8, 16),
        use_layer_norm=True,
    )
    tcn2 = TCN(
        nb_filters=nb_filters,
        nb_stacks = 1,
        kernel_size=10,
        dilations=(1,2, 4, 8, 16,32),
        use_layer_norm=True,
    )
    tcn3 = TCN(
        nb_filters=nb_filters,
        nb_stacks = 1,
        kernel_size=10,
        dilations=(2, 4, 8, 16,32,64),
        use_layer_norm=True,
    )
    x = tcn1(x_input)
    print(f'TCN.receptive_field: {tcn1.receptive_field}.')
    x = tcn2(x_input)
    print(f'TCN.receptive_field: {tcn2.receptive_field}.')
    x = tcn3(x_input)
    print(f'TCN.receptive_field: {tcn3.receptive_field}.')
    
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(output_sh, activation='sigmoid', name='dense')(x)
    
    model = Model(inputs=x_input, outputs=x, name='csi_model')
    return model

In [46]:
model=csi_network_inc_res((5, 5), 7)
def log_metrics(epoch, logs):

    wandb.log(logs, step=epoch)

wandb_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_metrics)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit( X_train,y_train, batch_size=100,epochs=3,validation_data=[X_val, y_val], callbacks=[
        tf.keras.callbacks.ModelCheckpoint('/home/vijay/paper_jc/conference/models/ph1_tcn.hdf5',
                                            monitor='val_accuracy',
                                            save_best_only=True,
                                            save_weights_only=False),
        wandb_callback
    ])

TCN.receptive_field: 559.
TCN.receptive_field: 1135.
TCN.receptive_field: 2269.
Epoch 1/3
11825/11825 [==============================] - 271s 22ms/step - loss: 0.1646 - accuracy: 0.9514 - val_loss: 0.0313 - val_accuracy: 0.9914
Epoch 2/3
   52/11825 [..............................] - ETA: 3:23 - loss: 0.0603 - accuracy: 0.9835

wandb: WARNING (User provided step: 0 is less than current step: 4. Dropping entry: {'loss': 0.16464577615261078, 'accuracy': 0.9514063000679016, 'val_loss': 0.031344857066869736, 'val_accuracy': 0.991386890411377, '_timestamp': 1722243341.2507956}).


11825/11825 [==============================] - 262s 22ms/step - loss: 0.0371 - accuracy: 0.9901 - val_loss: 0.0162 - val_accuracy: 0.9955
Epoch 3/3
   28/11825 [..............................] - ETA: 3:59 - loss: 0.0253 - accuracy: 0.9950

wandb: WARNING (User provided step: 1 is less than current step: 4. Dropping entry: {'loss': 0.03706025332212448, 'accuracy': 0.9900820255279541, 'val_loss': 0.016223786398768425, 'val_accuracy': 0.9955323338508606, '_timestamp': 1722243603.6235664}).


11825/11825 [==============================] - 269s 23ms/step - loss: 0.0224 - accuracy: 0.9941 - val_loss: 0.0086 - val_accuracy: 0.9976


wandb: WARNING (User provided step: 2 is less than current step: 4. Dropping entry: {'loss': 0.022441569715738297, 'accuracy': 0.9941200613975525, 'val_loss': 0.008613589219748974, 'val_accuracy': 0.9976202845573425, '_timestamp': 1722243872.160073}).
